<a href="https://colab.research.google.com/github/ddiestra/avila-dataset/blob/master/Ajuste_de_Hiperparametros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Ajuste de Hiperparametros

##Lectura de Datos

In [1]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00459/avila.zip
!unzip avila.zip

--2020-07-06 20:06:00--  https://archive.ics.uci.edu/ml/machine-learning-databases/00459/avila.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 584600 (571K) [application/x-httpd-php]
Saving to: ‘avila.zip’

avila.zip           100%[===================>] 570.90K  1.70MB/s    in 0.3s    

2020-07-06 20:06:00 (1.70 MB/s) - ‘avila.zip’ saved [584600/584600]

Archive:  avila.zip
   creating: avila/
  inflating: avila/avila-tr.txt      
  inflating: avila/avila-ts.txt      
  inflating: avila/avila-description.txt  


In [4]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

In [14]:
file_name = "avila/avila-tr.txt"
column_names = ['intercolumnar_distance', 'upper_margin', 'lower_margin', 'exploitation', 'row_number', 'modular_ratio', 'interlinear_spacing', 'weight', 'peak_number', 'mr_is', 'class'];
dataset = pd.read_csv(file_name,  header=None, names=column_names)
labelencoder.fit(dataset['class']);
dataset['class'] = labelencoder.transform(dataset['class'])

In [15]:
file_test = "avila/avila-ts.txt"
dataset_test = pd.read_csv(file_test,  header=None, names=column_names)
dataset_test['class'] = labelencoder.transform(dataset_test['class'])
dataset.head()

,intercolumnar_distance,upper_margin,lower_margin,exploitation,row_number,modular_ratio,interlinear_spacing,weight,peak_number,mr_is,class
0,0.266074,-0.165620,0.320980,0.483299,0.172340,0.273364,0.371178,0.929823,0.251173,0.159345,0
1,0.130292,0.870736,-3.210528,0.062493,0.261718,1.436060,1.465940,0.636203,0.282354,0.515587,0
2,-0.116585,0.069915,0.068476,-0.783147,0.261718,0.439463,-0.081827,-0.888236,-0.123005,0.582939,0
3,0.031541,0.297600,-3.210528,-0.583590,-0.721442,-0.307984,0.710932,1.051693,0.594169,-0.533994,0
4,0.229043,0.807926,-0.052442,0.082634,0.261718,0.148790,0.635431,0.051062,0.032902,-0.086652,5


In [16]:
Y = dataset['class']
X = dataset.drop('class', axis=1)
X.head()

,intercolumnar_distance,upper_margin,lower_margin,exploitation,row_number,modular_ratio,interlinear_spacing,weight,peak_number,mr_is
0,0.266074,-0.165620,0.320980,0.483299,0.172340,0.273364,0.371178,0.929823,0.251173,0.159345
1,0.130292,0.870736,-3.210528,0.062493,0.261718,1.436060,1.465940,0.636203,0.282354,0.515587
2,-0.116585,0.069915,0.068476,-0.783147,0.261718,0.439463,-0.081827,-0.888236,-0.123005,0.582939
3,0.031541,0.297600,-3.210528,-0.583590,-0.721442,-0.307984,0.710932,1.051693,0.594169,-0.533994
4,0.229043,0.807926,-0.052442,0.082634,0.261718,0.148790,0.635431,0.051062,0.032902,-0.086652


In [17]:
Yt = dataset_test['class']
Xt = dataset_test.drop('class', axis=1)
Xt.head()

,intercolumnar_distance,upper_margin,lower_margin,exploitation,row_number,modular_ratio,interlinear_spacing,weight,peak_number,mr_is
0,-3.498799,0.250492,0.232070,1.224178,-4.922215,1.145386,0.182426,-0.165983,-0.123005,1.087144
1,0.204355,-0.354049,0.320980,0.410166,-0.989576,-2.218127,0.220177,0.181844,2.090879,-2.009758
2,0.759828,-1.304042,-0.023991,-0.973663,-0.006417,-0.349509,-0.421580,-0.450127,0.469443,0.060952
3,-0.005490,0.360409,0.281860,-0.213479,-1.168333,-1.013906,-0.346080,1.176165,0.968347,-0.627999
4,0.080916,0.101320,0.104040,0.140490,0.261718,0.480988,0.710932,-0.253430,-0.497183,0.155681


##Eliminas algunas caracteristicas 
Basandonos en el colab anterior decidimos usar solo 5 de las caracteristicas 

In [9]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif, mutual_info_classif

In [18]:
filtrado = SelectKBest(mutual_info_classif, k=5).fit(X, Y)
X_new = filtrado.transform(X)
X_new[:5]

array([[ 0.266074, -0.16562 ,  0.32098 ,  0.483299,  0.17234 ],
       [ 0.130292,  0.870736, -3.210528,  0.062493,  0.261718],
       [-0.116585,  0.069915,  0.068476, -0.783147,  0.261718],
       [ 0.031541,  0.2976  , -3.210528, -0.58359 , -0.721442],
       [ 0.229043,  0.807926, -0.052442,  0.082634,  0.261718]])

In [19]:
Xt_new = filtrado.transform(Xt)
Xt_new[:5]

array([[-3.498799,  0.250492,  0.23207 ,  1.224178, -4.922215],
       [ 0.204355, -0.354049,  0.32098 ,  0.410166, -0.989576],
       [ 0.759828, -1.304042, -0.023991, -0.973663, -0.006417],
       [-0.00549 ,  0.360409,  0.28186 , -0.213479, -1.168333],
       [ 0.080916,  0.10132 ,  0.10404 ,  0.14049 ,  0.261718]])

##Entrenamos cada modelo

In [20]:
from sklearn.model_selection import GridSearchCV

###Decision Tree

Decission Tree Base

In [37]:
dt = DecisionTreeClassifier()
dt.fit(X_new, Y)
print('Max Depth: ', dt.tree_.max_depth)
print('Score: ',dt.score(Xt_new,Yt))
print('F1: ', f1_score(Yt, dt.predict(Xt_new), average='weighted'))

Max Depth:  17
Score:  0.9992334962153876
F1:  0.9992545918949806


Busqueda por Grilla

In [32]:
params = {"max_depth": [10,11,12,13,14,15,16,17, None], "max_features": [3,4,5], "min_samples_leaf": [3,4,5,6,7], "criterion": ["gini", "entropy"]}
tree = DecisionTreeClassifier()
dt_cv = GridSearchCV(tree, params, cv=5)
dt_cv.fit(X_new, Y)
print("Mejores Parametros: {}".format(dt_cv.best_params_))
print("Mejor Score {}".format(dt_cv.best_score_))

Mejores Parametros: {'criterion': 'entropy', 'max_depth': 17, 'max_features': 4, 'min_samples_leaf': 3}
Mejor Score 0.9975071907957813


Reentrenamos como los parametros encontrados

In [38]:
dt_final = DecisionTreeClassifier(max_depth=17, max_features=4, min_samples_leaf=3, criterion='entropy')
dt_final.fit(X_new, Y)
print('Score: ',dt_final.score(Xt_new,Yt))
print('F1: ', f1_score(Yt, dt_final.predict(Xt_new), average='weighted'))

Score:  0.9994251221615407
F1:  0.9994426717770257


Encontramos una mejora de 0.00018

###Random Forest

Random Forest Base

In [39]:
rt = RandomForestClassifier(40, n_jobs=-1, oob_score=True)
rt.fit(X_new, Y)
print('Score: ',dt.score(Xt_new,Yt))
print('F1: ', f1_score(Yt, rt.predict(Xt_new), average='weighted'))

Score:  0.9992334962153876
F1:  0.9994426717770257


Busqueda por Grilla

In [41]:
params = {"n_estimators": [40,50,100,150,200], "max_depth": [10,11,12,13,14,15,16,17], "max_features": [3,4,5], "min_samples_leaf": [3,4,5,6,7]}
forest = RandomForestClassifier(oob_score=True)
rf_cv = GridSearchCV(forest, params, cv=5)
rf_cv.fit(X_new, Y)
print("Mejores Parametros: {}".format(rf_cv.best_params_))
print("Mejor Score {}".format(rf_cv.best_score_))

Mejores Parametros: {'max_depth': 14, 'max_features': 3, 'min_samples_leaf': 3, 'n_estimators': 40}
Mejor Score 0.9978906999041227


In [43]:
rf_final = RandomForestClassifier(oob_score=True, max_depth=14, max_features=3, min_samples_leaf=3, n_estimators=40)
rf_final.fit(X_new, Y)
print('Score: ',rf_final.score(Xt_new,Yt))
print('F1: ', f1_score(Yt, rf_final.predict(Xt_new), average='weighted'))

Score:  0.9994251221615407
F1:  0.9994426717770257


El F1 score no mejore, aunque si el accuracy, y llegamos a los mismos valores que el Decission Tree

###KNN

KNN BASE

In [44]:
knn = KNeighborsClassifier(4)
knn.fit(X_new, Y)
print('Score: ',knn.score(Xt_new,Yt))
print('F1: ', f1_score(Yt, knn.predict(Xt_new), average='weighted'))

Score:  0.9989460572961579
F1:  0.9987117932315139


In [46]:
params = {"n_neighbors": [3,4,5, 6,8,10], "weights": ['uniform', 'distance'], "metric": ['euclidean', 'manhattan']}
forest = KNeighborsClassifier()
knn_cv = GridSearchCV(forest, params, cv=5)
knn_cv.fit(X_new, Y)
print("Mejores Parametros: {}".format(knn_cv.best_params_))
print("Mejor Score {}".format(knn_cv.best_score_))

Mejores Parametros: {'metric': 'euclidean', 'n_neighbors': 4, 'weights': 'distance'}
Mejor Score 0.998849472674976


In [48]:
knn_final = KNeighborsClassifier(n_neighbors=4, metric='euclidean',weights='distance')
knn_final.fit(X_new, Y)
print('Score: ',knn_final.score(Xt_new,Yt))
print('F1: ', f1_score(Yt, knn_final.predict(Xt_new), average='weighted'))

Score:  0.999137683242311
F1:  0.9988998935928342
